<a href="https://www.kaggle.com/code/yutodennou/lgbm-and-nn-mohs-hardness?scriptVersionId=154394729" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🗃️Import Libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import tensorflow_probability as tfp

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


# 📏Preprocessing

In [2]:
train_dataset = pd.read_csv('/kaggle/input/playground-series-s3e25/train.csv')

In [3]:
features = ['allelectrons_Total', 'density_Total', 'allelectrons_Average',
            'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
            'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
            'zaratio_Average', 'density_Average', 'Hardness']

In [4]:
X = train_dataset[features].drop(columns='Hardness')
y = train_dataset.Hardness

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model_pre = LGBMRegressor(max_depth=5,n_estimators=250)
model_pre.fit(X, y)
X_new = X.copy()
X_new['Hardness_pred'] = model_pre.predict(X)

In [7]:
def loss_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=50)

def metric_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0)

In [8]:
callbacks_list1 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
    tf.keras.callbacks.TerminateOnNaN()
] 
callbacks_list2 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00002),
    tf.keras.callbacks.TerminateOnNaN()
] 
callbacks_list3 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00003),
    tf.keras.callbacks.TerminateOnNaN()
] 

In [9]:
def create_model(hp_learning_rate):
    
    input_layer = tf.keras.Input(shape=(len(features), ))
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    output_layer = tf.keras.layers.Dense(1)(x)    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp_learning_rate, beta_1=0.5),
                  loss=loss_fn,
                  metrics=metric_fn)
    
    return model

# 🔍Learning

In [10]:
model1 = create_model(0.013)
history1 = model1.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list1,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 7s 6ms/step - loss: 1.3589 - metric_fn: 4.9045 - val_loss: 1.2846 - val_metric_fn: 3.8911 - lr: 0.0130
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8428 - metric_fn: 3.2656 - val_loss: 0.8340 - val_metric_fn: 3.3456 - lr: 0.0130
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7855 - metric_fn: 3.0585 - val_loss: 0.6908 - val_metric_fn: 3.1845 - lr: 0.0130
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7181 - metric_fn: 3.0517 - val_loss: 0.9542 - val_metric_fn: 3.6502 - lr: 0.0130
Epoch 5/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6213 - metric_fn: 3.0643 - val_loss: 0.6053 - val_metric_fn: 3.2470 - lr: 0.0130
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6137 - metric_fn: 3.0696 - val_loss: 0.5628 - val_metric_fn: 3.2583 - lr: 0.0130
Epoch 7/100
293/293 [==============================]

In [11]:
model2 = create_model(0.009)
history2 = model2.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=150,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list2,
                    validation_split=0.1)

Epoch 1/150
293/293 [==============================] - 4s 5ms/step - loss: 1.2127 - metric_fn: 4.4190 - val_loss: 0.5954 - val_metric_fn: 4.1157 - lr: 0.0090
Epoch 2/150
293/293 [==============================] - 1s 5ms/step - loss: 0.8381 - metric_fn: 3.9165 - val_loss: 0.6237 - val_metric_fn: 5.8346 - lr: 0.0090
Epoch 3/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7873 - metric_fn: 3.5358 - val_loss: 0.5505 - val_metric_fn: 4.3584 - lr: 0.0090
Epoch 4/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7175 - metric_fn: 3.3832 - val_loss: 0.5604 - val_metric_fn: 4.5899 - lr: 0.0090
Epoch 5/150
293/293 [==============================] - 1s 5ms/step - loss: 0.6795 - metric_fn: 3.2659 - val_loss: 0.5799 - val_metric_fn: 3.8268 - lr: 0.0090
Epoch 6/150
293/293 [==============================] - 1s 5ms/step - loss: 0.6539 - metric_fn: 3.1461 - val_loss: 0.6116 - val_metric_fn: 3.4801 - lr: 0.0090
Epoch 7/150
293/293 [==============================]

In [12]:
model3 = create_model(0.010)
history3 = model3.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=150,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list3,
                    validation_split=0.1)

Epoch 1/150
293/293 [==============================] - 4s 5ms/step - loss: 1.3538 - metric_fn: 4.9434 - val_loss: 0.7337 - val_metric_fn: 3.8476 - lr: 0.0100
Epoch 2/150
293/293 [==============================] - 1s 5ms/step - loss: 0.8745 - metric_fn: 3.3200 - val_loss: 0.7145 - val_metric_fn: 3.2397 - lr: 0.0100
Epoch 3/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7993 - metric_fn: 3.2241 - val_loss: 0.6140 - val_metric_fn: 3.4245 - lr: 0.0100
Epoch 4/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7322 - metric_fn: 2.9595 - val_loss: 0.7225 - val_metric_fn: 3.5584 - lr: 0.0100
Epoch 5/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7160 - metric_fn: 3.0545 - val_loss: 0.6821 - val_metric_fn: 3.2800 - lr: 0.0100
Epoch 6/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7114 - metric_fn: 2.9591 - val_loss: 0.5226 - val_metric_fn: 3.0793 - lr: 0.0100
Epoch 7/150
293/293 [==============================]

In [13]:
model4 = create_model(0.010)
history4 = model4.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list1,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 4s 5ms/step - loss: 1.2453 - metric_fn: 4.3627 - val_loss: 0.8761 - val_metric_fn: 3.5445 - lr: 0.0100
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8553 - metric_fn: 3.4493 - val_loss: 0.7658 - val_metric_fn: 3.3958 - lr: 0.0100
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7934 - metric_fn: 3.1269 - val_loss: 0.5543 - val_metric_fn: 3.1963 - lr: 0.0100
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7259 - metric_fn: 3.0547 - val_loss: 0.5028 - val_metric_fn: 3.2361 - lr: 0.0100
Epoch 5/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6284 - metric_fn: 3.0396 - val_loss: 0.9092 - val_metric_fn: 3.5482 - lr: 0.0100
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.5795 - metric_fn: 3.0484 - val_loss: 0.5457 - val_metric_fn: 3.2611 - lr: 0.0100
Epoch 7/100
293/293 [==============================]

In [14]:
model5 = create_model(0.012)
history5 = model5.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list2,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 4s 5ms/step - loss: 1.4120 - metric_fn: 5.8494 - val_loss: 1.3515 - val_metric_fn: 5.9051 - lr: 0.0120
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8800 - metric_fn: 3.6639 - val_loss: 0.8506 - val_metric_fn: 3.3727 - lr: 0.0120
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7408 - metric_fn: 3.0730 - val_loss: 0.6057 - val_metric_fn: 3.0889 - lr: 0.0120
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7993 - metric_fn: 3.4951 - val_loss: 0.5551 - val_metric_fn: 3.5679 - lr: 0.0120
Epoch 5/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6553 - metric_fn: 3.4610 - val_loss: 0.5405 - val_metric_fn: 3.4293 - lr: 0.0120
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6016 - metric_fn: 3.2795 - val_loss: 0.5290 - val_metric_fn: 3.5206 - lr: 0.0120
Epoch 7/100
293/293 [==============================]

In [15]:
model6 = create_model(0.013)
history6 = model6.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list3,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 4s 5ms/step - loss: 1.2751 - metric_fn: 4.1584 - val_loss: 0.9280 - val_metric_fn: 3.2304 - lr: 0.0130
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8147 - metric_fn: 3.0187 - val_loss: 0.6559 - val_metric_fn: 3.0141 - lr: 0.0130
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6984 - metric_fn: 3.0053 - val_loss: 0.7159 - val_metric_fn: 3.1427 - lr: 0.0130
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6214 - metric_fn: 3.1713 - val_loss: 0.5000 - val_metric_fn: 3.4650 - lr: 0.0130
Epoch 5/100
293/293 [==============================] - 1s 5ms/step - loss: 0.5946 - metric_fn: 3.2210 - val_loss: 0.4938 - val_metric_fn: 3.3533 - lr: 0.0130
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.5978 - metric_fn: 3.2117 - val_loss: 0.5342 - val_metric_fn: 3.3314 - lr: 0.0130
Epoch 7/100
293/293 [==============================]

In [16]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e25/sample_submission.csv')
test_dataset = pd.read_csv('/kaggle/input/playground-series-s3e25/test.csv')

In [17]:

test_dataset['Hardness_pred'] = model_pre.predict(test_dataset.astype('float32').drop(columns='id'))
test_dataset["Hardness"] = model1.predict(test_dataset.astype('float32').drop(columns='id'))*0.3+model2.predict(test_dataset.astype('float32').drop(columns='id'))*0.2+model3.predict(test_dataset.astype('float32').drop(columns='id'))*0.1+model4.predict(test_dataset.astype('float32').drop(columns='id'))*0.2+model5.predict(test_dataset.astype('float32').drop(columns='id'))*0.1+model6.predict(test_dataset.astype('float32').drop(columns='id'))*0.1
sample_submission = test_dataset[['id', "Hardness"]]
sample_submission.to_csv("submission.csv", index=False)

217/217 [==============================] - 0s 1ms/step


In [18]:
sample_submission

,id,Hardness
0,10407,2.590769
1,10408,2.444890
2,10409,5.755329
3,10410,3.428776
4,10411,5.750341
...,...,...
6934,17341,5.750032
6935,17342,3.944494
6936,17343,5.746502
6937,17344,5.540216
